In [ ]:
import numpy as np
import pandas as pd
import nltk

In [ ]:
Movies = pd.read_csv("data/tmdb_5000_movies.csv")
Credits = pd.read_csv("data/tmdb_5000_credits.csv")

In [ ]:
movies = Movies.merge(Credits , on ="title")

In [ ]:
movies = movies[["movie_id" , "genres" , "title" , "overview" , "keywords" , "crew" , "cast"  ]]

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
import ast
def conv(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l


In [ ]:
movies["genres"]=movies["genres"].apply(conv)
movies["keywords"] = movies["keywords"].apply(conv)


In [ ]:
def conv2(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i["job"] == "Director":
            l.append(i["name"])
    return l

In [ ]:
movies["crew"] = movies["crew"].apply(conv2)

In [ ]:
def conv3(obj):
    x = ast.literal_eval(obj)
    return [i["name"] for i in x[:3]]

In [ ]:
def find_bad_row():
    for idx, obj in enumerate(movies["cast"]):
        x = ast.literal_eval(obj)
        if len(x) < 3:
            return idx, x
    return "No bad rows"

find_bad_row()

In [ ]:
movies["cast"] = movies["cast"].apply(conv3)

In [ ]:
movies["overview"] = movies["overview"].apply(lambda x : x.split())

In [ ]:
movies["genres"] =  movies["genres"].apply(lambda x : [i.replace(" " , "") for i in x])
movies["cast"] =  movies["cast"].apply(lambda x : [i.replace(" " , "") for i in x])
# movies["overview"] =  movies["overview"].apply(lambda x : [i.replace(" " , "") for i in x])
movies["crew"] =  movies["crew"].apply(lambda x : [i.replace(" " , "") for i in x])
movies["keywords"] =  movies["keywords"].apply(lambda x : [i.replace(" " , "") for i in x])
movies.head()
# Data cleaned successfully

In [ ]:
#  now final df for model 
movies["tags"] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]

In [ ]:
new_df = movies[["title" , "movie_id" , "tags"]]
new_df.head()

In [ ]:
new_df["tags"] = new_df["tags"].apply(lambda x : (" ".join(x)).lower())


In [ ]:
# Vectorisation  Data is cleaned and ready for our model 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 , stop_words="english")

In [ ]:
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(txt):
    y=[]
    for i in txt.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df["tags"].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
# main function 
def recommend(movies):
    movies_index = new_df[new_df["title"] == movies].index[0]
    distance = similarity[movies_index]
    movies_list = sorted(list(enumerate(distance)), reverse=True , key=lambda x : x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
                        

In [ ]:
recommend('Avatar')